In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix as confmat
from sklearn.metrics import f1_score as f1
# from sklearn.metrics import plot_confusion_matrix as plot_confmat

# import re
# import string
import random
import importlib

# import nltk
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer

import torch
import preprocess
import dan

import gensim
# load dataset and preprocess
twt = pd.read_csv('train.csv')
twt = twt.set_index('id')
twt.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## 0. Data Preprocessing

### 0.1 Data Cleaning

In [7]:
text = twt['text'].to_list()
prep_text = [preprocess.processing(i) for i in text]
print('{} tweets lowered, tokenized, alphanumerized, stop-stripped, and stemmed.'.format(len(prep_text)))
twt['prepped'] = prep_text
twt.head()

7613 tweets lowered, tokenized, alphanumerized, stop-stripped, and stemmed.


,keyword,location,text,target,prepped
id,,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth..."
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]"
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[all, residents, asked, to, shelter, in, place..."
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfires, evacuation, order..."
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[just, got, sent, this, photo, from, ruby, ala..."


### 0.2 Embedding

In [9]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

twt['vec'] = pd.Series([preprocess.tweet_vec(tweet, word2vec) for tweet in twt['prepped']], index = twt.index)

devtrain_idx = twt.loc[~twt['vec'].isna()].index.tolist()
random.shuffle(devtrain_idx)

train_p = 0.7
train_idx = devtrain_idx[:round(train_p*len(devtrain_idx))]
dev_idx = devtrain_idx[round(train_p*len(devtrain_idx)):]

train = [(vec, targ) for targ, vec in  zip(twt['target'][train_idx], twt['vec'][train_idx])]
dev = [(vec, targ) for targ, vec in  zip(twt['target'][dev_idx], twt['vec'][dev_idx])]

## 1. NN construction

In [12]:
import importlib
importlib.reload(dan)
net = dan.Net()
net.train(train, dev)

[1,     1] loss: 0.691	Dev FI: 0.000
[1,     2] loss: 0.651	Dev FI: 0.272
[2,     1] loss: 0.621	Dev FI: 0.690
[2,     2] loss: 0.585	Dev FI: 0.732
[3,     1] loss: 0.560	Dev FI: 0.713
[3,     2] loss: 0.542	Dev FI: 0.703
[4,     1] loss: 0.530	Dev FI: 0.700
[4,     2] loss: 0.529	Dev FI: 0.717
[5,     1] loss: 0.523	Dev FI: 0.725
[5,     2] loss: 0.513	Dev FI: 0.736
[6,     1] loss: 0.521	Dev FI: 0.737
[6,     2] loss: 0.500	Dev FI: 0.739
[7,     1] loss: 0.518	Dev FI: 0.738
[7,     2] loss: 0.490	Dev FI: 0.742
[8,     1] loss: 0.514	Dev FI: 0.741
[8,     2] loss: 0.483	Dev FI: 0.749
[9,     1] loss: 0.512	Dev FI: 0.752
[9,     2] loss: 0.474	Dev FI: 0.749
[10,     1] loss: 0.510	Dev FI: 0.750
[10,     2] loss: 0.467	Dev FI: 0.749
Finished Training.


## Dev Performance and Error Analysis

In [32]:
# dev[0]
ys, y_stars = net.get_eval_data(dev)
# confmat(ys, y_stars)
f1(ys, y_stars)

twt['pred'] = pd.Series(y_stars, index = dev_idx)


# for error analysis
twt[['keyword', 'location', 'text', 'prepped','vec', 'target','pred']].to_csv('error_analysis.csv')

## Create Test Submissions

In [35]:
# create vector of mean of all word vectors
mean_vec = np.zeros((300,1))
for vec_targ in train:
    mean_vec = np.add(mean_vec, vec_targ[0])
mean_vec = mean_vec/len(train)

# read in test data and preprocess tweets
test = pd.read_csv('test.csv')
proc_text = [preprocess.processing(i) for i in text]
targets = np.zeros((len(test), 1))
print('{} tweets read from test.csv'.format(test.shape[0]))


# test_data = [(tweet_vec(x, word2vec), y) if for x, y in zip(proc_test, targets)]
test['vec'] = pd.Series([preprocess.tweet_vec(tweet, word2vec) for tweet in test['text'].tolist()], index = test.index)

print('{} tweets processed in the test set.'.format(test.shape[0]))

test_data = test['vec'].tolist()


_, y_stars = net.get_eval_data(test_data, mode = 'test')

# create columns for submission data
id = test['id'].to_numpy()
target = y_stars

# create df with submission data and write to csv
submission = pd.DataFrame({'id': id, 'target': target})
submission.set_index('id').to_csv('submission.csv')

# also save analysis csv for reviewing decisions
test['prepped'] = proc_text
test['pred'] = target
test[['keyword', 'location', 'text', 'prepped','pred']].to_csv('test_error_analysis.csv')

3263 tweets read from test.csv
3263 tweets processed in the test set.
3263 3263


In [43]:
test[['id', 'pred']].set_index('id').to_csv('submission.csv')

## 2. Naive Bayes with Bag of Words

In [44]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# create raw bow text and labels, shuffle
bow = twt['text'].to_numpy()
labels = twt['target'].to_numpy()

shuffle_idx = [x for x in range(len(bow))]
random.shuffle(shuffle_idx)

bow = bow[shuffle_idx]
labels = labels[shuffle_idx]

# train bag-of-words
count_vect = CountVectorizer()
x_bow = count_vect.fit(bow)

# create train dev split
train_bow = bow[:round(train_p*len(bow))]
dev_bow = bow[round(train_p*len(bow)):]
train_bow = count_vect.transform(train_bow)
dev_bow = count_vect.transform(dev_bow)

train_bow_labels = labels[:round(train_p*len(bow))]
dev_bow_lables = labels[round(train_p*len(bow)):]

print('Created {} bag-of-words vectorizations.'.format(train_bow.shape[0] + dev_bow.shape[0]))
clf = MultinomialNB().fit(train_bow, train_bow_labels)
print('Naive Bayes training complete.')

# get dev performances
ys = clf.predict(dev_bow)

# dev performance
print('Naive Bayes dev performance {}'.format(f1(ys, dev_bow_lables)))

Created 7613 bag-of-words vectorizations.
Naive Bayes training complete.
Naive Bayes dev performance 0.7500000000000001


## Missingness Exploration

We suspect that location data can be useful for predicting disasters. However, we have a sizable proportion of the data for which the location data is missing. In order to address this issue, we needed to first identify whether the data is missing completely at random, missing at random, or not missing at random. For the data to be MCAR, the mechanism of missingness of the data must be independent of all other observed features of the data. These other observed features, in our case, include the text of the tweet. Reviewing the Twitter policy on tweet-level location data, we found that twitter users must opt in to location services to embed location data in there tweets, as well as intentionally include location data on each desired tweet. This policy gives us an insight into some mechanisms of missingness - users forget that they can include location data, or the intentionally elect not to for some tweets. This second mechanism would preclude categorization as missing completely at random. Furthermore, as users of social media, the authors have firsthand experience with the relationship between the location from which a tweet was sent, and the inclusion of location data. Twitter users may want to inform followers that they are tweeting from an impressive or otherwise unusual location - a famous concert venue, or a historic city, or even the site of a terrible natural disaster - a decision surely not made at random. Being Missing Not a Random, our options for mitigating location data missingness are few. We could drop the data altogether, but decided that appending it to the contents of the tweet would be an acceptable approach that prevents information loss.


## Hyperparameter Tuning

In [29]:
# hidden Layer dimensions

pre_opts = [(1, 0, 0), (1, 1, 0), (1, 0, 1) , (1, 1, 1), (0, 1, 0), (0, 1, 1), (0, 0, 1)]
Fs_ls = []
for lower, stopwords, stem in pre_opts:
    prep_text = [preprocess.processing(i, lower, stopwords, stem) for i in text]
    # print('{} tweets lowered, tokenized, alphanumerized, stop-stripped, and stemmed.'.format(len(prep_text)))

    twt['vec'] = pd.Series([preprocess.tweet_vec(tweet, word2vec) for tweet in prep_text], index = twt.index)

    devtrain_idx = twt.loc[~twt['vec'].isna()].index.tolist()
    random.shuffle(devtrain_idx)

    train_p = 0.7
    train_idx = devtrain_idx[:round(train_p*len(devtrain_idx))]
    dev_idx = devtrain_idx[round(train_p*len(devtrain_idx)):]

    train = [(vec, targ) for targ, vec in  zip(twt['target'][train_idx], twt['vec'][train_idx])]
    dev = [(vec, targ) for targ, vec in  zip(twt['target'][dev_idx], twt['vec'][dev_idx])]

    Fs = []
    for hiddenDim in range(2,302,25):
        # print('Executing model with hidden layer dimensionality {}'.format(hiddenDim))
        net = dan.Net(hiddenDim)
        net.train(train, dev, verbose = False)
        ys, y_stars = net.get_eval_data(dev)
        Fs.append(f1(ys, y_stars))
    Fs_ls.append(Fs)

results = {i : Fs_ls[i] for i in range(len(Fs_ls))}
pd.DataFrame(results).to_csv('crossvalidation.csv')

7613 tweets lowered, tokenized, alphanumerized, stop-stripped, and stemmed.
Executing model with hidden layer dimensionality 2
Finished Training.
Executing model with hidden layer dimensionality 27
Finished Training.
Executing model with hidden layer dimensionality 52
Finished Training.
Executing model with hidden layer dimensionality 77
Finished Training.
Executing model with hidden layer dimensionality 102
Finished Training.
Executing model with hidden layer dimensionality 127
Finished Training.
Executing model with hidden layer dimensionality 152
Finished Training.
Executing model with hidden layer dimensionality 177
Finished Training.
Executing model with hidden layer dimensionality 202
Finished Training.
Executing model with hidden layer dimensionality 227
Finished Training.
Executing model with hidden layer dimensionality 252
Finished Training.
Executing model with hidden layer dimensionality 277
Finished Training.
7613 tweets lowered, tokenized, alphanumerized, stop-stripped, an

In a grid search dev evaluation of the hiddenDim and preprocessing hyperparameters, we found that hiddenDim = 52, lowercasing, and stopword inclusion provided the best results, at F1 = 0.744